# Test Scientist: Run an Experiment

In [1]:
from simulator.simulation.scientists import ExperimentRunner, ExperimentCollater
from simulator.models import SimExperiment
from winds.generators import OscillatoryGenerator
from commons.utilities import list_model_fields

import numpy as np
from numpy import pi

import requests

## Load or Create Winds

Specify the wind parameters we want to use and load the corresponding winds trajectory if it exists, otherwise create it. We'll use the HTTP interface to do this, issuing commands via the appropriate *url* endpoints...

### WindGenParams:

In [9]:
# wind velocity oscillating in XY-plane (Circle)
windgenparams = {
    'is_oscillatory': True,
    'base_velocity': np.array([0,0,0]), # not currently used for oscillatory
    'amplitude': np.array([1,1,0]),
    'frequency': np.array([1,1,1]),
    'phase_offset': np.array([0,np.pi/2,0]),
}

res = requests.post('http://localhost:8000/winds/wind-gen-params/', data=windgenparams)
res_data = res.json()
if res.status_code == 409:
    message = res_data['message']
    print(message)
else:
    print('Created.')
windgenparams_obj_id = res_data['id']
print(windgenparams_obj_id)

Already exists
26b671a4-cfe2-4931-829e-e7a780a48abb


### WindSpacetime:

In [10]:
windspacetimeparams = {
    'generator_name': 'oscillatory',
    'generator_params': windgenparams_obj_id,
    'duration': 100,
    'timestep': .01,
}

res = requests.post('http://localhost:8000/winds/wind-spacetimes/', data=windspacetimeparams)
res_data = res.json()
windspacetime_obj_id = res_data['id']
print(res_data)
print(windspacetime_obj_id)

{'message': 'Already exists', 'id': '7bed9515-0a22-4516-a6b1-4f19923df74c'}
7bed9515-0a22-4516-a6b1-4f19923df74c


Great, we now have the UUID for a windspacetime specified by the parameters above. We now pass this id as a parameter to the scientist used to run the experiment.

## Run Experiment

We'll call the scientist class directly until we setup an HTTP view to encapsulate this logic.

In [11]:
params = {
        'windgenparams_id': windgenparams_obj_id,
        'windspacetime_id': windspacetime_obj_id,
        'num_trials': 10,
        'timestep': 0.01,
        'prob_speed_fn_name': 'Uniform',
        'max_initial_speed': 42, # m/s
        'prob_speed_center': None,
        'prob_speed_spread': None,
        'prob_timing_fn_name': 'Uniform',
        'max_wait_time': 1, # s
        'prob_timing_center': None,
        'prob_timing_spread': None,
        'prob_aiming_fn_name': 'Uniform',
        'prob_aiming_geometry': 'Spherical',
        'prob_aiming_X1_min': -pi/4,  # -45 deg
        'prob_aiming_X1_max': pi/4,   #  45 deg
        'prob_aiming_X1_center': None,
        'prob_aiming_X1_spread': None,
        'prob_aiming_X2_min': pi/6,   # 30 deg down from vertical 
        'prob_aiming_X2_max': 2*pi/3, # 120 deg down from vertical = 30 deg below horiztonal
        'prob_aiming_X2_center': None,
        'prob_aiming_X2_spread': None,
        'prob_aiming_X3_min': 0,      # throwaway coordinate
        'prob_aiming_X3_max': 1,
        'prob_aiming_X3_center': None,
        'prob_aiming_X3_spread': None,
}
runner = ExperimentRunner(params)
simtrial_ids = runner.run_experiment()

[Scientist] >>>>>>>>>>>>>> Running Trial #0 >>>>>>>>>>>>>>
[SimTrialRunner] Preparing parameters...
[SimTrialRunner] OK.
[SimTrialRunner] ====================================
[SimTrialRunner] Run parameters are:
  >> t_initial    = 73
  >> p_initial    = [ 0  0 10]
  >> v_initial    = [3.06032786 2.41638494 0.89606314]
  >> timestep     = 0.01
  >> g            = 9.81
  >> m            = 0.0456
  >> drag_coef    = 0
  >> verbosity    = 1
[SimTrialRunner] ====================================
[SimTrialRunner] Running trial...
[SimTrialRunner] Initializing ball trajectory...
[SimTrialRunner] Initialized.
[SimTrialRunner] ====================================
[SimTrialRunner] Completed Run.
[SimTrialRunner]  >> Ball hit ground?.. YES.
[SimTrialRunner]  >> Trajectory duration = 2.27s.
[SimTrialRunner] Interpolating for final position between...
[SimTrialRunner]   >> p1: [5.15852879 2.19479218 0.10405997]
[SimTrialRunner]   >> p2: [ 5.1903854   2.19903488 -0.0360914 ]
[SimTrialRunner] Interpo

## Collate and Save Experiment

In [12]:
chunked_simtrial_ids = [simtrial_ids,]
collater = ExperimentCollater(params, chunked_simtrial_ids=chunked_simtrial_ids)
se_obj = collater.save_experiment()
se_obj_id = se_obj.id.__str__()
print(se_obj_id)

bbef6815-747e-440b-bc41-d6cae4d3bfee


### Check saved experiment details

In [13]:
se_obj_check = SimExperiment.objects.get(pk=se_obj_id)
print(se_obj_check.num_trials)
print(se_obj_check.simtrials.all().count())
print(se_obj_check.prob_aiming_geometry)

10
10
Spherical


## Chunkify the Experiment Run

Let's break the workload of computing `num_trials` SimTrials into chunks run across `num_workers` workers. For now, we'll do it "in series" for a quick test, but eventually we can convert this to "in parallel".

In [23]:
# add num_workers to params
params['num_workers'] = 2

# determine chunk size: runs_per_worker
num_trials = params['num_trials']
num_workers = params['num_workers']
runs_per_worker = int(num_trials/num_workers) # assume divisibility for now

# do runs
chunked_simtrial_ids = []
for _ in range(num_workers):
    worker_params = params.copy()
    worker_params['num_trials'] = runs_per_worker
    runner = ExperimentRunner(worker_params)
    simtrial_ids = runner.run_experiment()
    chunked_simtrial_ids.append(simtrial_ids)
    
print("chunked_simtrial_ids:")
print(chunked_simtrial_ids)

# collate chunks to single experiment
collater = ExperimentCollater(params, chunked_simtrial_ids=chunked_simtrial_ids)
se_obj = collater.save_experiment()

print("SimExperiment id:", se_obj)

[Scientist] >>>>>>>>>>>>>> Running Trial #0 >>>>>>>>>>>>>>
[SimTrialRunner] Preparing parameters...
[SimTrialRunner] OK.
[SimTrialRunner] ====================================
[SimTrialRunner] Run parameters are:
  >> t_initial    = 45
  >> p_initial    = [ 0  0 10]
  >> v_initial    = [14.87609632  5.64865592 -4.35125429]
  >> timestep     = 0.01
  >> g            = 9.81
  >> m            = 0.0456
  >> drag_coef    = 0
  >> verbosity    = 1
[SimTrialRunner] ====================================
[SimTrialRunner] Running trial...
[SimTrialRunner] Initializing ball trajectory...
[SimTrialRunner] Initialized.
[SimTrialRunner] ====================================
[SimTrialRunner] Completed Run.
[SimTrialRunner]  >> Ball hit ground?.. YES.
[SimTrialRunner]  >> Trajectory duration = 1.52s.
[SimTrialRunner] Interpolating for final position between...
[SimTrialRunner]   >> p1: [16.5695898   6.24622444  0.07492299]
[SimTrialRunner]   >> p2: [16.71786133  6.30580117 -0.07159455]
[SimTrialRunner] I